1. Number of Characters (Length of the string)

In [1]:
## Compute the number of characters
text = "I don't know"
num_char = len(text)

df['num_char'] = df['reviews'].apply(len)

2. Number of Words

In [3]:
text = "Maru has a little lamb"
word = text.split()
print(len(word))

5


In [ ]:
def word_count(string):
    word = string.split()
    
    return len(word)

df['num_words'] = df['reviews'].apply(word_count)

## AVERAGE WORD LENGTH

In [ ]:
def avg_word_length(x):
    words = x.split()
    word_length = [len(word) for word in words]
    avg_length = sum(word_length)/len(words)
    return avg_length

df['avg_word_length'] = df['reviews'].apply(avg_word_length)

## HASH TAGS AND MENTIONS (FOR TWEETS MOSTLY)

In [ ]:
def hashtag_count(string):
    words = string.split()
    hashtags = [word for word in words if word.startswith('#')]
    return len(hashtags)

tweets['mention_count'].hist()
plt.title('Mention count distribution')
plt.show()


In [ ]:
## number of sentences
## number of paragraphs
## number of words starting with an uppercase

## READABILITY TESTS

In [ ]:
## Flesch reading ease
## Gunning fog index
## Simple measure of Goobledygook (SMOG)
## Dale-Chale score

In [6]:
# !pip install textatistic
## import textatistic library
from textatistic import Textatistic

You should consider upgrading via the 'c:\users\osuntoki\anaconda3\python.exe -m pip install --upgrade pip' command.


In [ ]:
## Create a textatistic object
readability_scores = Textatistic(text).scores

## Generate scores
print(readability_scores['flesch_score'])
print(readability_scores['gunningfog_score'])
print(readability_scores['fleschkincaid_score'])
print(readability_scores['smog_score'])
print(readability_scores['dalechall_score'])
print(readability_scores['sybl_count'])
print(readability_scores['sent_count'])
print(readability_scores['word_count'])
print(readability_scores['polysyblword_count']) ## no of words with three or more syllables
print(readability_scores['notdalechall_count']) ## number of words not on Dale-Chall list of words understood by 80% of 4th graders
print(readability_scores['char_count']) ## number of non-space characters

## Text Preprocessing Techniques

1. Converting Words into lowercase
2. Removing leading and trailing whitespaces
3. Removing pucntuation
4. Removing stopwords
5. Expanding contractions
6. Removing special characters such as numbers, emojis


In [21]:
## Tokenization 
# We use the spaCy library

import spacy
import en_core_web_sm

## load the en_core_web_sm model
nlp = spacy.load('en_core_web_sm')

## initailize string
string = "Hello! I don't want to see you."

## create a doc object
doc = nlp(string)

## Generate list of tokens
tokens = [token.text for token in doc]
print(tokens)

['Hello', '!', 'I', 'do', "n't", 'want', 'to', 'see', 'you', '.']


In [23]:
## Lemmatization - Converting a word into its base form
lemmas = [token.lemma_ for token in doc]
print(lemmas)

['hello', '!', 'I', 'do', "n't", 'want', 'to', 'see', 'you', '.']


In [24]:
print(' '.join(lemmas))

hello ! I do n't want to see you .


In [27]:
## Text Cleaning - Removing non-alphabetic characters
a_lemmas = [lemma for lemma in lemmas
           if lemma.isalpha()]
print(' '.join(a_lemmas))

hello I do want to see you


In [28]:
## Removing stopwords - Words that occur so commonly
stopwords = spacy.lang.en.stop_words.STOP_WORDS

a_lemmas = [lemma for lemma in lemmas
           if lemma.isalpha() and lemma not in stopwords]
print(' '.join(a_lemmas))

hello I want


In [ ]:
# Function to preprocess text
def preprocess(text):
  	# Create Doc object
    doc = nlp(text, disable=['ner', 'parser'])
    # Generate lemmas
    lemmas = [token.lemma_ for token in doc]
    # Remove stopwords and non-alphabetic characters
    a_lemmas = [lemma for lemma in lemmas 
            if lemma.isalpha() and lemma not in stopwords]
    
    return ' '.join(a_lemmas)
  
# Apply preprocess to ted['transcript']
ted['transcript'] = ted['transcript'].apply(preprocess)
print(ted['transcript'])

## PART OF SPEECH TAGGING

In [29]:
## Generate list of tokens and pos tags
pos = [(token.text, token.pos_) for token in doc]
print(pos)

[('Hello', 'INTJ'), ('!', 'PUNCT'), ('I', 'PRON'), ('do', 'AUX'), ("n't", 'PART'), ('want', 'VERB'), ('to', 'PART'), ('see', 'VERB'), ('you', 'PRON'), ('.', 'PUNCT')]


In [ ]:
nlp = spacy.load('en_core_web_sm')

# Returns number of proper nouns
def proper_nouns(text, model=nlp):
  	# Create doc object
    doc = model(text)
    # Generate list of POS tags
    pos = [token.pos_ for token in doc]
    
    # Return number of proper nouns
    return pos.count('PROPN')

print(proper_nouns("Abdul, Bill and Cathy went to the market to buy apples.", nlp))

In [ ]:
nlp = spacy.load('en_core_web_sm')

# Returns number of other nouns
def nouns(text, model=nlp):
  	# Create doc object
    doc = model(text)
    # Generate list of POS tags
    pos = [token.pos_ for token in doc]
    
    # Return number of other nouns
    return pos.count('NOUN')

In [ ]:
headlines['num_noun'] = headlines['title'].apply(nouns)

# Compute mean of other nouns
real_noun = headlines[headlines['label'] == 'REAL']['num_noun'].mean()
fake_noun = headlines[headlines['label'] == 'FAKE']['num_noun'].mean()

# Print results
print("Mean no. of other nouns in real and fake headlines are %.2f and %.2f respectively"%(real_noun, fake_noun))


headlines['num_propn'] = headlines['title'].apply(proper_nouns)

# Compute mean of proper nouns
real_propn = headlines[headlines['label'] == 'REAL']['num_propn'].mean()
fake_propn = headlines[headlines['label'] == 'FAKE']['num_propn'].mean()

# Print results
print("Mean no. of proper nouns in real and fake headlines are %.2f and %.2f respectively"%(real_propn, fake_propn))


## NAMED ENTITY RECOGNITION 
- Identifying and classifying named entities into predefined categories

Categories include person, organization, country



In [30]:
# Load the required model
nlp = spacy.load('en_core_web_sm')

# Create a Doc instance 
text = 'Sundar Pichai is the CEO of Google. Its headquarters is in Mountain View.'
doc = nlp(text)

# Print all named entities and their labels
for ent in doc.ents:
    print(ent.text, ent.label_)

Sundar Pichai PERSON
Google ORG
Mountain View GPE


In [ ]:
def find_persons(text):
  # Create Doc object
  doc = nlp(text)
  
  # Identify the persons
  persons = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
  
  # Return persons
  return persons

## VECTORIZATION - BUILDING A BAG OF WORDS MODEL

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer = CountVectorizer(strip_accents = 'ascii', stop_words = 'english', lowercase = False)

In [38]:
## Generate matrix of word vectors
import pandas as pd
import numpy as np
corpus = pd.Series(['The Lion is the king of the jungle', 'Lions have lifespan of a decade', 'The lion is an endangered specie'])

bow_matrix = vectorizer.fit_transform(corpus)
print(bow_matrix.toarray())

[[1 0 1 0 0 1 1 0 0 0]
 [0 1 0 1 0 0 0 1 0 0]
 [0 0 1 0 1 0 0 0 1 1]]


In [ ]:
# Create CountVectorizer object
vectorizer = CountVectorizer()

# Generate matrix of word vectors
bow_matrix = vectorizer.fit_transform(corpus)

# Convert bow_matrix into a DataFrame
bow_df = pd.DataFrame(bow_matrix.toarray())

# Map the column names to vocabulary 
bow_df.columns = vectorizer.get_feature_names()

# Print bow_df
print(bow_df)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['message'],df['labels'], test_size = 0.25)

X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

In [ ]:
## Taining a naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_bow, y_train)

accuracy = clf.score(X_test_bow, y_test)
print(accuracy)

In [40]:
## BUILDING N-GRAM MODELS

## In bag of words, context is lost

""" Application of n-grams include: sentence completion, spelling correction, machine translation correction """

' Application of n-grams include: sentence completion, spelling correction, machine translation correction '

In [ ]:
vectorizer = CountVectorizer(strip_accents = 'ascii', stop_words = 'english', lowercase = False, ngram_range = (1,3))


# Generate n-grams upto n=1
vectorizer_ng1 = CountVectorizer(ngram_range=(1,1))
ng1 = vectorizer_ng1.fit_transform(corpus)

# Generate n-grams upto n=2
vectorizer_ng2 = CountVectorizer(ngram_range=(1,2))
ng2 = vectorizer_ng2.fit_transform(corpus)

# Generate n-grams upto n=3
vectorizer_ng3 = CountVectorizer(ngram_range=(1,3))
ng3 = vectorizer_ng3.fit_transform(corpus)

# Print the number of features for each model
print("ng1, ng2 and ng3 have %i, %i and %i features respectively" % (ng1.shape[1], ng2.shape[1], ng3.shape[1]))




# Define an instance of MultinomialNB 
clf_ng = MultinomialNB()

# Fit the classifier 
clf_ng.fit(X_train_ng, y_train)

# Measure the accuracy 
accuracy = clf_ng.score(X_test_ng, y_test)
print("The accuracy of the classifier on the test set is %.3f" % accuracy)

# Predict the sentiment of a negative review
review = "The movie was not good. The plot had several holes and the acting lacked panache."
prediction = clf_ng.predict(ng_vectorizer.transform([review]))[0]
print("The sentiment predicted by the classifier is %i" % (prediction))

In [41]:
## TERM FREQUENCY - INVERSE DOCUMENT FREQUENCY (TF_IDF)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer = TfidfVectorizer(stop_words = 'english')

tfidf_matrix = vectorizer.fit_transform(corpus)
print(tfidf_matrix.toarray())

[[0.         0.         0.         0.         0.25434658 0.33443519
  0.33443519 0.         0.25434658 0.         0.25434658 0.
  0.76303975]
 [0.         0.46735098 0.         0.46735098 0.         0.
  0.         0.46735098 0.         0.46735098 0.35543247 0.
  0.        ]
 [0.45954803 0.         0.45954803 0.         0.34949812 0.
  0.         0.         0.34949812 0.         0.         0.45954803
  0.34949812]]


## COSINE SIMILARITY

In [ ]:
## import cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

## used mostly in recommenders

In [43]:
""" Cosine similarity and linear kernel performs essentially the same thing, but linear_kernel is faster"""

' Cosine similarity and linear kernel performs essentially the same thing, but linear_kernel is faster'

In [ ]:
# Record start time
start = time.time()

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix,tfidf_matrix)

# Print cosine similarity matrix
print(cosine_sim)

# Print time taken
print("Time taken: %s seconds" %(time.time() - start))



# Record start time
start = time.time()

# Compute cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Print cosine similarity matrix
print(cosine_sim)

# Print time taken
print("Time taken: %s seconds" %(time.time() - start))

In [ ]:
# Generate mapping between titles and index
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

def get_recommendations(title, cosine_sim, indices):
    # Get index of movie that matches title
    idx = indices[title]
    # Sort the movies based on the similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar movies
    sim_scores = sim_scores[1:11]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]


# Initialize the TfidfVectorizer 
tfidf = TfidfVectorizer(stop_words='english')

# Construct the TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(movie_plots)

# Generate the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
 
# Generate recommendations 
print(get_recommendations("The Dark Knight Rises", cosine_sim, indices))

In [75]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
nlp = spacy.load('en_core_web_sm')
vectorizer = TfidfVectorizer(stop_words = 'english')
doc1 = nlp("I am happy")
doc2 = nlp('I am sad')
doc3 = nlp('I am joyous')
corpus = pd.Series(['I am happpy','I am sad', 'I am joyous'])

tfidf_matrix1 = vectorizer.fit_transform(corpus)
#tfidf_matrix2 = vectorizer.fit_transform(doc2)
#tfidf_matrix3 = vectorizer.fit_transform(doc3)

cosine_sim = cosine_similarity(tfidf_matrix1, tfidf_matrix1)

In [76]:
tfidf_matrix1.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

## WORD EMBEDDINGS

In [6]:


import spacy
nlp = spacy.load('en_core_web_lg')
# Create the doc object
doc = nlp("happy joyous sad")

# Compute pairwise similarity scores
for token1 in doc:
  for token2 in doc:
    print(token1.text, token2.text, token1.similarity(token2))

happy happy 1.0
happy joyous 0.5333031
happy sad 0.64389884
joyous happy 0.5333031
joyous joyous 1.0
joyous sad 0.43832767
sad happy 0.64389884
sad joyous 0.43832767
sad sad 1.0
